# How to generate states
1.  “Layer 1 Input”: The variable x n (t) to “the DDQN is size 2K + 2 vector”. Initial K + 1
input entries indicate the action taken at time t − 1. Specifically, if the user has not
transmitted at time slot t − 1, the first entrance is set to 1 and the subsequent K entries are
set to 0. The following K input entries are the capacity of each channel. The last input is 1
if ACK signal has been received. Else, if communication has failed or no communication
has been made, and it is set to 0.

Since i have to feed the states in the DQN , first I created state for an user
![alt text](state_input_dqn.png "Title")
This is the state format for a user in one time slot in the environment with 3 users and 2 channels

In [1]:
# imports
from multi_user_network_env import env_network
import numpy as np
import matplotlib.pyplot as plt

In [2]:
NUM_USERS = 3
NUM_CHANNELS = 2
ATTEMPT_PROB = 1

In [3]:
#initializing the environment
env = env_network(NUM_USERS,NUM_CHANNELS,ATTEMPT_PROB)

In [5]:
#It creates a one hot vector of a number as num with size as len
def one_hot(num,len):
    assert num >=0 and num < len ,"error"
    vec = np.zeros([len],np.int32)
    vec[num] = 1
    return vec

In [6]:
one_hot(5,6)  

array([0, 0, 0, 0, 0, 1], dtype=int32)

One-hot vector of ```len (6)``` having ```value (5)```

In [7]:
# generates states from action and obs as returned from environment
def state_generator(action,obs):
    input_vector = []
    if action is None:
        print ('None')
        sys.exit()
    for user_i in range(action.size):
        input_vector_i = one_hot(action[user_i],NUM_CHANNELS+1)
        channel_alloc = obs[-1]
        input_vector_i = np.append(input_vector_i,channel_alloc)
        input_vector_i = np.append(input_vector_i,int(obs[user_i][0]))    #ACK
        input_vector.append(input_vector_i)
    return input_vector

In [8]:
# user 1 and 2 takes action 1 and user 3 takes action 2 
action = np.array([1,1,2])
obs = env.step(action)
print obs

[(0, 0.0), (0, 0.0), (1, 1.0), array([1, 0], dtype=int32)]


In [11]:
states = state_generator(action, obs)
print states
assert len(states) == NUM_USERS

[array([0, 1, 0, 1, 0, 0]), array([0, 1, 0, 1, 0, 0]), array([0, 0, 1, 1, 0, 1])]


```[array([0, 1, 0, 1, 0, 0])```
```array([0, 1, 0, 1, 0, 0])```
```array([0, 0, 1, 1, 0, 1])]```

Each array represents states for each user in th format given above

Since there are 3 users , ```state_generator(action, obs)``` returned 3 arrays

Rest of the training is in ```train.py``` file in this repository